# TEXT ANALYTICS GROUP ASSIGNMENT #2

#### Group Members: Brooks Beckelman, Zack Bilderback, Dallas Griffin, Estevan Gonzalez, Sean Kessel, Davis Townsend

In [1]:
import pandas as pd
from pandas import Series, DataFrame
%pylab inline

import seaborn

Populating the interactive namespace from numpy and matplotlib


In [3]:
yelp = pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')

In [4]:
# Convert rating to high (4,5) or low (1,2,3)
yelp['rating'] = np.where(yelp['stars'] >= 4, 'high','low')
yelp.head(3)

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,rating
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This location is out of business. I drove by i...,low
1,2,2,2,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,= = = = = = CLOSED = = = = = =This JB s locati...,low
2,4,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,This is just a basic (albeit mini) chain greas...,high


## Task A

In [5]:
from sklearn.cross_validation import train_test_split

train, test = train_test_split(yelp, test_size=0.25, random_state=123)

In [6]:
x_train = train.drop(['stars', 'Review', 'rating'], axis=1)
y_train = train['rating']

x_test = test.drop(['stars', 'Review', 'rating'], axis=1)
y_test = test['rating']

### Naive Bayes

In [7]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb_classifier = bnb.fit(x_train, y_train)
bnb_predict = bnb.predict(x_test)
bnb_accuracy = bnb.score(x_test, y_test)

print 'Bernoulli Naive Bays Accuracy:', bnb_accuracy

Bernoulli Naive Bays Accuracy: 0.6816


In [8]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb_classifier = mnb.fit(x_train, y_train)
mnb_predict = mnb.predict(x_test)
mnb_accuracy = mnb.score(x_test, y_test)

print 'Multinomial Naive Bayes Accuracy:', mnb_accuracy

Multinomial Naive Bayes Accuracy: 0.6812


## Task B

In [12]:
train_b = train[['Review','rating']]

In [14]:
import random 
rows = random.sample(train_b.index, 6000)
df_6000 = train_b.ix[rows]

In [16]:
import math
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [28]:
descriptions = []
pay = []

for index, value in df_6000.iterrows():
    descriptions.append(value['Review'])
    pay.append(value['rating'])

In [32]:
#Split data into test/training set
trainset_size = int(round(len(df_6000)*0.75))

X_train = np.array([''.join(el) for el in descriptions[0:trainset_size]])
y_train = np.array([el for el in pay[0:trainset_size]])

X_test = np.array([''.join(el) for el in descriptions[trainset_size+1:len(descriptions)]]) 
y_test = np.array([el for el in pay[trainset_size+1:len(pay)]]) 

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = 'english',   \
                             max_features = None)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

#Run Naive Bayes Classifier
nb_classifier = MultinomialNB().fit(X_train, y_train)

y_nb_predicted = nb_classifier.predict(X_test)

print 'Precision:' + str(metrics.precision_score(y_test, y_nb_predicted, average = 'binary', pos_label='high'))
print 'Recall: ' + str(metrics.recall_score(y_test, y_nb_predicted, average = 'binary', pos_label='high'))
print 'Accuracy: ' + str(metrics.accuracy_score(y_test, y_nb_predicted))

Precision:0.838624338624
Recall: 0.936945812808
Accuracy: 0.835223482322


### Preprocessing for any tokenization

In [17]:
tokenizer = RegexpTokenizer(r'\w+') #Tokenizes and removes punctuation

#Strip and decode FullDescription column so we can tokenize it
df_6000['tokenized_strip'] = df_6000.apply(lambda row: row['Review'].decode('utf-8').strip(),axis=1)
df_6000['tokenized'] = df_6000.apply(lambda row: tokenizer.tokenize(row['tokenized_strip']),axis=1)

In [23]:
#Combine each row of tokens into one list
tokens_series = pd.Series(df_6000['tokenized'])
tokens_list = tokens_series.tolist()

#Remove stop words and make all tokens lower case
tokens = []
for sentence in tokens_list:
    for word in sentence:
        if word.lower() not in stopwords.words('english'):
            tokens.append(word.lower())